In [28]:
# Import dependencies
import pandas as pd
import numpy as np
import time
import re
from langdetect import detect, detect_langs
from nltk.stem.snowball import SnowballStemmer

In [29]:
# Create DataFrame from CSV file
nlp_df = pd.read_csv('../../Data/nlp_df.csv')
nlp_df.head(3)

,song,song_id,artist,artist_id,category,category_id,popularity,genres,audio_ft_danceability,audio_ft_energy,...,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words,words,filtered,language
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,8,93,"['dance', 'pop']",0.392,0.574,...,0.145,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,1,"['', 'im', 'like', 'the', 'water', 'when', 'yo...","['', 'im', 'like', 'water', 'ship', 'rolled', ...","(1, 'en')"
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,8,78,"['indie', 'pop', 'electropop']",0.581,0.584,...,0.366,0.756,179.954,206046.0,4.0,Didnt even wanna go out whyd you call me ? Iv...,1,"['', 'didnt', 'even', 'wanna', 'go', 'out', 'w...","['', 'didnt', 'even', 'wanna', 'go', 'whyd', '...","(1, 'en')"
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,8,95,['australian'],0.662,0.413,...,0.134,0.467,93.005,161385.0,4.0,You cut out a piece of me and now I bleed int...,1,"['', 'you', 'cut', 'out', 'a', 'piece', 'of', ...","['', 'cut', 'piece', 'bleed', 'internally', 'l...","(1, 'en')"


In [30]:
# Create a list of all words, unique word counts, and filtered words
t0 = time.time()
words_list = []
unique_word_counts = []
filtered_words_list = []
for index, row in nlp_df.iterrows():
    filtered_words = row['filtered']
    filtered_words = filtered_words.replace(',', '').replace("'", '')
    filtered_words = filtered_words.replace('[', '').replace(']', '')
    filtered_words = filtered_words.replace('#', '').replace('&nbsp', '')
    filtered_words = filtered_words.replace('?', '? ').replace(',', '')
    filtered_words = filtered_words.replace('/', ' ').replace('‚', '')
    filtered_words = filtered_words.replace('\\u200a', '').replace('\\u200b', '')
    filtered_words = filtered_words.replace('\\u2063', '').replace('\u202f', '')
    filtered_words = filtered_words.replace('\\u2028', ' ').replace('\\u2008', ' ')
    while ('\\u200e' in filtered_words) or ('\\xa0' in filtered_words):
        filtered_words = filtered_words.replace('\\u200e', '')
        filtered_words = filtered_words.replace('\\xa0', '')
    filtered_words_list.append(filtered_words)
    unique_words = list(set(filtered_words.strip().split(' ')))
    unique_word_counts.append(len(unique_words))
    words_list.extend(unique_words)
word_columns = list(set(words_list))
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
len(word_columns)

Run time: 1.660233974456787 seconds


39220

In [31]:
# Add a unique_word_count column and replace the filtered column
nlp_df['unique_word_count'] = unique_word_counts
nlp_df['filtered'] = filtered_words_list
nlp_df.head(1)

,song,song_id,artist,artist_id,category,category_id,popularity,genres,audio_ft_danceability,audio_ft_energy,...,audio_ft_valence,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words,words,filtered,language,unique_word_count
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,8,93,"['dance', 'pop']",0.392,0.574,...,0.529,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,1,"['', 'im', 'like', 'the', 'water', 'when', 'yo...",im like water ship rolled night rough surface...,"(1, 'en')",87


In [32]:
# Remove songs with less than 25 unique words
nlp_df = nlp_df[nlp_df['unique_word_count']>=25]
len(nlp_df)

8113

In [33]:
# Update word list
words_list = []
for index, row in nlp_df.iterrows():
    filtered_words = row['filtered']
    unique_words = list(set(filtered_words.strip().split(' ')))
    words_list.extend(unique_words)
word_columns = list(set(words_list))
len(word_columns)

39072

# Try to remove non English songs

In [34]:
# Find songs indeces with one language that is not English
one_lang_not_en = []
mult_langs_not_en = []
for index, row in nlp_df.iterrows():
    if (row['language'][1] == '1') & (row['language'][5:7] != 'en'):
        one_lang_not_en.append(index)
    if (row['language'][1] != '1') & (row['language'][5:7] != 'en'):
        mult_langs_not_en.append(index)
print(len(one_lang_not_en))
len(mult_langs_not_en)

99


103

In [35]:
# Print non English songs with one language
for index in one_lang_not_en:
    print(index)
    print(nlp_df.loc[index, :][['song', 'artist']])
    print(nlp_df.loc[index, :]['lyrics'][0:65])
songs_to_keep = [78, 452, 715, 878, 1633, 2150, 2402, 2407, 2427, 2526, 2835, 2979, 3232,
                 3359, 3791, 4368, 4643, 5692, 6328, 6379, 6387, 6523, 6722, 7853]
len(songs_to_keep)

78
song      Keep It Simple (feat. Wilder Woods)
artist                                 Matoma
Name: 78, dtype: object
 We lost our love Somewhere in the darkness over yonder Somewhere
140
song      Mango Negro
artist          Rubio
Name: 140, dtype: object
   Sintiendo que me voy Mis pies flotan En el aire al estar Parec
161
song           Melocotón
artist    Kimberley Tell
Name: 161, dtype: object
 Deja que se evapore el ego Y haz eco De lo que nace del deseo Mm
275
song      Baka Baka
artist    emir taha
Name: 275, dtype: object
 Bu duvarlara baka baka Duvarlara baka baka Bu duvarlara baka bak
706
song      Laisse moi (feat. Hamza)
artist         Lous and The Yakuza
Name: 706, dtype: object
   Juste laisse moi laisse moi laisse laisse laisse Juste laisse 
715
song      Whoopty
artist         CJ
Name: 715, dtype: object
 Heavy on the SSO shit man Loyalty over royalty yall niggas know 
878
song      Combination Pizza Hut and Taco Bell
artist                             Das Racist
Name

Name: 5709, dtype: object
O mio babbino caro Mi piace è bello bello Voandare in Porta Rossa
5724
song      Sogno di Volare (“The Dream of Flight”)
artist                            Christopher Tin
Name: 5724, dtype: object
 Una Volta che Avrai Spiccato il Volo Deciderai Sguardo Verso il 
5726
song      Delibes: Les Filles de Cadix
artist                     Léo Delibes
Name: 5726, dtype: object
Nous venions de voir le taureau Trois garçons trois fillettes Sur
5742
song      O Mio Babbino Caro
artist       Giacomo Puccini
Name: 5742, dtype: object
In Italiano  O mio babbino caro mi piace è bello bello Voandare i
5743
song      Vois sur ton chemin
artist          Les Choristes
Name: 5743, dtype: object
 Vois sur ton chemin Gamins oubliés égarés Donne leur la main Pou
5744
song      La Mamma Morta
artist      Maria Callas
Name: 5744, dtype: object
La mamma morta mhanno Alla porta della stanza mia Moriva e mi sal
5747
song           Jojo's Theme
artist    Michael Giacchino
Name: 5747, dtyp

24

In [36]:
# Print non English songs with multiple languages
for index in mult_langs_not_en:
    print(index)
    print(nlp_df.loc[index, :][['song', 'artist']])
    print(nlp_df.loc[index, :]['lyrics'][0:65])
songs_to_remove = [264, 924, 1610, 1618, 2299, 2368, 3688, 5482, 5656, 5658, 5688, 5695, 
                   5710, 5715, 5719, 5720, 5750, 5792, 6385]
len(songs_to_remove)

21
song      My Head & My Heart
artist               Ava Max
Name: 21, dtype: object
 Baby now and then I think about me now and who I could have been
34
song         Fever
artist    Dua Lipa
Name: 34, dtype: object
 Before you came around I was doing just fine Usually usually usu
109
song      Something Just Like This
artist            The Chainsmokers
Name: 109, dtype: object
 Ive been reading books of old The legends and the myths Achilles
175
song      Take You Dancing
artist        Jason Derulo
Name: 175, dtype: object
 Da da da da da da Da da da da da da Da da da da da da Da da da d
182
song      Breaking Me
artist          Topic
Name: 182, dtype: object
 La la la la la la la la   Call me what you wanna Ill be what you
239
song      when the party's over
artist            Billie Eilish
Name: 239, dtype: object
 Dont you know Im no good for you ? Ive learned to lose you cant 
264
song      Life Goes On
artist             BTS
Name: 264, dtype: object
   어느 날 세상이 멈췄어 아무런 예고도 하나 없이 봄

Name: 5695, dtype: object
 Pokarekare ana Nga wai o waiapu Whiti atu koe hine Marino ana e 
5710
song      Now We Are Free
artist       Lisa Gerrard
Name: 5710, dtype: object
Anol shedom Anol sheh lay konnah de veh um shaddai Flavum Nohm de
5715
song      Under the Stars
artist        Hans Zimmer
Name: 5715, dtype: object
 One two three four Pa da da dip du du dap Pa da da di da Pa da d
5719
song         God of War
artist    Bear McCreary
Name: 5719, dtype: object
Útlægr Guð Smán föður Von móður Þraut barnsins  Útlægr Guð Smán f
5720
song        Dragonborn
artist    Jeremy Soule
Name: 5720, dtype: object
Dovahkiin Dovahkiin naal ok zin los vahriin Wah dein vokul mahfae
5750
song           Requiem: Dies irae
artist    Wolfgang Amadeus Mozart
Name: 5750, dtype: object
Complete Structure of the Süssmayr Version of Wolfgang Amadeus Mo
5774
song       The Road Home
artist    Stephen Paulus
Name: 5774, dtype: object
 Ooo ooo ooo ooo Ooo ooo ooo ooo ooo   Tell me where is the road 
5792
song 

19

In [37]:
# Remove non English songs
songs_to_remove.extend([item for item in one_lang_not_en if item not in songs_to_keep])
nlp_df = nlp_df.drop(songs_to_remove)
print(len(songs_to_remove))
len(nlp_df)

104


8010

# Update word list

In [38]:
# Update word list
words_list = []
for index, row in nlp_df.iterrows():
    filtered_words = row['filtered']
    unique_words = list(set(filtered_words.strip().split(' ')))
    words_list.extend(unique_words)
word_columns = list(set(words_list))
len(word_columns)

34928

In [39]:
# Remove non alphabetic words from the word list
non_alpha_words = []
for word in word_columns:
    if not re.match(r'[a-zA-Z0-9?!/$&+]+$', word):
        non_alpha_words.append(word)
        word_columns.remove(word)
print(len(non_alpha_words))
len(word_columns)

447


34481

In [40]:
# Add column of word stems
t0 = time.time()
word_stems = []
stemmer = SnowballStemmer("english")
for index, row in nlp_df.iterrows():
    stems = []
    filtered_words = row['filtered']
    #for word in filtered_words.split():
    for word in filtered_words.strip().split(' '):
        stems.append(stemmer.stem(word))
    word_stems.append(stems)
nlp_df['word_stems'] = word_stems
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
nlp_df.head(3)

Run time: 23.236400842666626 seconds


,song,song_id,artist,artist_id,category,category_id,popularity,genres,audio_ft_danceability,audio_ft_energy,...,audio_ft_tempo,audio_ft_duration_ms,audio_ft_time_signature,lyrics,non_alpha_words,words,filtered,language,unique_word_count,word_stems
0,willow,0lx2cLdOt3piJbcaXIV74f,Taylor Swift,06HL4z0CvFAxyc27GXpf02,pop,8,93,"['dance', 'pop']",0.392,0.574,...,81.112,214707.0,4.0,Im like the water when your ship rolled in th...,1,"['', 'im', 'like', 'the', 'water', 'when', 'yo...",im like water ship rolled night rough surface...,"(1, 'en')",87,"[im, like, water, ship, roll, night, rough, su..."
1,Stay Next To Me (with Chelsea Cutler),6SGG5AxHShqSYiV9fCWpZz,Quinn XCII,3ApUX1o6oSz321MMECyIYd,pop,8,78,"['indie', 'pop', 'electropop']",0.581,0.584,...,179.954,206046.0,4.0,Didnt even wanna go out whyd you call me ? Iv...,1,"['', 'didnt', 'even', 'wanna', 'go', 'out', 'w...",didnt even wanna go whyd call ? ive long day...,"(1, 'en')",104,"[didnt, even, wanna, go, whyd, call, ?, , ive,..."
2,WITHOUT YOU,27OeeYzk6klgBh83TSvGMA,The Kid LAROI,2tIP7SsRs7vjIcLrU85W8J,pop,8,95,['australian'],0.662,0.413,...,93.005,161385.0,4.0,You cut out a piece of me and now I bleed int...,1,"['', 'you', 'cut', 'out', 'a', 'piece', 'of', ...",cut piece bleed internally left without witho...,"(1, 'en')",55,"[cut, piec, bleed, intern, left, without, with..."


In [41]:
# Create list of unique word stems
stems_list = []
for index, row in nlp_df.iterrows():
    stems = row['word_stems']
    unique_stems = list(set(stems))
    stems_list.extend(unique_stems)
stem_columns = list(set(stems_list))
len(stem_columns)

24442

In [43]:
# Remove non alphabetic words from the word list
non_alpha_words = []
for word in stem_columns:
    if not re.match(r'[a-zA-Z0-9?!/$&+]+$', word):
        non_alpha_words.append(word)
        stem_columns.remove(word)
print(len(non_alpha_words))
len(stem_columns)

438


24004

# Create new DataFrame for ML model

In [47]:
# Create DataFrame for the ML model
t0 = time.time()
columns = ['song_name', 'artist_name', 'category_name', 'category_id', 'genre_list', 
           'audio_ft_danceability', 'audio_ft_energy', 'audio_ft_key', 
           'audio_ft_loudness', 'audio_ft_mode', 'audio_ft_speechiness', 
           'audio_ft_acousticness', 'audio_ft_instrumentalness', 'audio_ft_liveness',
           'audio_ft_valence', 'audio_ft_tempo', 'audio_ft_duration_ms', 
           'audio_ft_time_signature']
columns.extend(stem_columns)
lyric_SF_df = pd.DataFrame(columns=columns)
lyric_SF_df['song_name'] = nlp_df['song']
lyric_SF_df['artist_name'] = nlp_df['artist']
lyric_SF_df['category_name'] = nlp_df['category']
lyric_SF_df['category_id'] = nlp_df['category_id']
lyric_SF_df['genre_list'] = nlp_df['genres']
lyric_SF_df['audio_ft_danceability'] = nlp_df['audio_ft_danceability']
lyric_SF_df['audio_ft_energy'] = nlp_df['audio_ft_energy']
lyric_SF_df['audio_ft_key'] = nlp_df['audio_ft_key']
lyric_SF_df['audio_ft_loudness'] = nlp_df['audio_ft_loudness']
lyric_SF_df['audio_ft_mode'] = nlp_df['audio_ft_mode']
lyric_SF_df['audio_ft_speechiness'] = nlp_df['audio_ft_speechiness']
lyric_SF_df['audio_ft_acousticness'] = nlp_df['audio_ft_acousticness']
lyric_SF_df['audio_ft_instrumentalness'] = nlp_df['audio_ft_instrumentalness']
lyric_SF_df['audio_ft_liveness'] = nlp_df['audio_ft_liveness']
lyric_SF_df['audio_ft_valence'] = nlp_df['audio_ft_valence']
lyric_SF_df['audio_ft_tempo'] = nlp_df['audio_ft_tempo']
lyric_SF_df['audio_ft_duration_ms'] = nlp_df['audio_ft_duration_ms']
lyric_SF_df['audio_ft_time_signature'] = nlp_df['audio_ft_time_signature']
lyric_SF_df = lyric_SF_df.fillna(0)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
lyric_SF_df.head(3)

Run time: 99.81258893013 seconds


,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,...,opul,scorch,no,dens,muslim,xception,heavyweight,cellmat,wholi,mozambiqu
0,willow,Taylor Swift,pop,8,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,...,0,0,0,0,0,0,0,0,0,0
1,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,...,0,0,0,0,0,0,0,0,0,0
2,WITHOUT YOU,The Kid LAROI,pop,8,['australian'],0.662,0.413,0.0,-7.357,1.0,...,0,0,0,0,0,0,0,0,0,0


In [48]:
# Add the term frequencies to the DataFrame
t0 = time.time()
for index, row in lyric_SF_df.iterrows():
    for stem in nlp_df['word_stems'][index]:
            try:
                lyric_SF_df.loc[index, stem]+=1
            except(Exception) as e:
                print(stem)
    if index % 100 == 0:
        t1 = time.time()
        print(f'Run time: {t1-t0} seconds')
        t0 = time.time()
lyric_SF_df.head(3)















Run time: 10.46939992904663 seconds












































































































































































































































































































































































































































































está
señorita


así
así


azúcar
¿no





sacúdelo
aquí
muévelo
muévelo
muévelo
así
sí








está
más


así


azúcar
¿no

























































































































































être
ça
médicament






ça
fièvr
ça
cœur
à




fièvr


fièvr
fièvr
fièvr


ça
fièvr
ça
cœur
à


























































































































falling￼



































































































































































































































































































































déjà
























































































































































curé
más
cuestión






























































































































































































내
그림자를
가른
한줄기
빛
어둡기만한
내
삶을
뒤집어
놓은
너
나
어쩌면
아무것도
아니지
너를
만나기전엔
그저
보잘것
없던
나
보잘것
없던
나
그전
내
삶은
다
하루를
대충
때우기에
급급했었잖아
우리의
낮
우리의
밤
그래
우리의
삶
서롤
지탱하는
벗
서로의
닻
















































































































































































































































































































































































































































































































ménage



























































































































































































































Run time: 58.23975086212158 seconds







































































guantánamo

































































dussé
























élysé




































moët

















































































































































































sötét
ő
megsajnált
eljött
közénk









































9′s
















masterpièc



ι



















































moët
























jhené



jhené









javonté













mgmt@topdawgmusiccom
@absoul




℗






























beyoncé



















































señorita


















rosé




















































cliché













































jenné

































































































¡vámono











































































































































































































dussé














































































Run time: 53.572136878967285 seconds


























































































































































































































































für

für







für























adió








































































































































































































































































































































































































































































































































































































































































































































































































...




























































































































































































































qué




tú


créeme


tú
muévelo






tú
muévelo
















































































































































pièg
décroché




contrarié
contrarié
contrarié
contrarié
brûle







































mâché










mâché
































































































































































































































































ово
јe
живот
élet
אלה
החיים
هذه
هي
الحياة
생명
生活
ชีวิต
それが人生




































































































































































~





































































































































~


















lovinâ€™



â©




























































































































































don´t

























































































































































































































































~









































































































































































































































































































































































































































































































































































déjà



















































































































































































































































































































































































Run time: 38.46480393409729 seconds

























































führer


über
über
über
über




über
über
über
über




über
über
über
über










































































































































































































































































































































































































































































































Run time: 41.39522194862366 seconds























tú
tú
está
último
llevará
tú
está
sería






tú
tú
está
último
llevará
tú
está
sería

































































































































































































































































































































































































































































































































































































































































































,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,...,opul,scorch,no,dens,muslim,xception,heavyweight,cellmat,wholi,mozambiqu
0,willow,Taylor Swift,pop,8,"['dance', 'pop']",0.392,0.574,7.0,-9.195,1.0,...,0,0,0,0,0,0,0,0,0,0
1,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,-4.928,1.0,...,0,0,0,0,0,0,0,0,0,0
2,WITHOUT YOU,The Kid LAROI,pop,8,['australian'],0.662,0.413,0.0,-7.357,1.0,...,0,0,0,0,0,0,0,0,0,0


In [49]:
# Remove columns of word stems that only appear once
t0 = time.time()
lyric_SF_df.loc[-1] = lyric_SF_df.sum(numeric_only=True)
lyric_SF_df.index = lyric_SF_df.index + 1  
lyric_SF_df = lyric_SF_df.sort_index()
lyric_SF_df = lyric_SF_df.drop(columns=lyric_SF_df.columns[(lyric_SF_df[0:1] == 1).any()])    
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
lyric_SF_df.head(3)

,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_loudness,audio_ft_mode,...,sim,devis,opul,scorch,no,dens,muslim,heavyweight,wholi,mozambiqu
0,NaN,NaN,NaN,53343.0,NaN,4515.3613,5346.5255,42464.0,-60295.899,5499.0,...,2.0,3.0,3.0,11.0,3.0,3.0,7.0,4.0,7.0,4.0
1,willow,Taylor Swift,pop,8.0,"['dance', 'pop']",0.3920,0.5740,7.0,-9.195,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"['indie', 'pop', 'electropop']",0.5810,0.5840,2.0,-4.928,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
# Create filtered DataFrame with columns of word stems that appear at least 4 times
t0 = time.time()
filtered_lyric_SF_df = lyric_SF_df.drop(columns=lyric_SF_df.columns[(lyric_SF_df[0:1]<4).any()])    
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
filtered_lyric_SF_df.head(3)

Run time: 0.6500930786132812 seconds


,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,...,ale,lack,slogan,libido,oop,scorch,muslim,heavyweight,wholi,mozambiqu
0,NaN,NaN,NaN,53343.0,NaN,4515.3613,5346.5255,42464.0,5499.0,663.5064,...,24.0,65.0,5.0,11.0,23.0,11.0,7.0,4.0,7.0,4.0
1,willow,Taylor Swift,pop,8.0,"['dance', 'pop']",0.3920,0.5740,7.0,1.0,0.1700,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"['indie', 'pop', 'electropop']",0.5810,0.5840,2.0,1.0,0.2840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [51]:
# Drop columns of word stems that are used in less than 4 songs
t0 = time.time()
filtered_lyric_SF_df.loc[-1] = filtered_lyric_SF_df.astype(bool).sum(axis=0)
filtered_lyric_SF_df.index = filtered_lyric_SF_df.index + 1
filtered_lyric_SF_df = filtered_lyric_SF_df.sort_index()
filtered_lyric_SF_df = filtered_lyric_SF_df.drop(columns=filtered_lyric_SF_df.columns[(filtered_lyric_SF_df[0:1]<4).any()])    
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
filtered_lyric_SF_df.head(3)

Run time: 4.6273839473724365 seconds


,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,...,cut,ale,lack,slogan,libido,oop,scorch,muslim,heavyweight,mozambiqu
0,8011,8011,8011,7588.0,8011,8011.0000,8011.0000,7148.0,5500.0,8011.0000,...,396.0,7.0,53.0,6.0,6.0,14.0,10.0,6.0,5.0,5.0
1,NaN,NaN,NaN,53343.0,NaN,4515.3613,5346.5255,42464.0,5499.0,663.5064,...,690.0,24.0,65.0,5.0,11.0,23.0,11.0,7.0,4.0,4.0
2,willow,Taylor Swift,pop,8.0,"['dance', 'pop']",0.3920,0.5740,7.0,1.0,0.1700,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
# Save DataFrames to CSV
t0 = time.time()
filtered_lyric_SF_df.to_csv('../../Data/filtered_lyric_SF.csv', index=False)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')

t0 = time.time()
lyric_SF_df.to_csv('../../Data/lyric_SF.csv', index=False)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')

Run time: 103.87505602836609 seconds
Run time: 170.53361701965332 seconds
